**Step 1: Import Libraries**

In [9]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

**Step 2: Set Up the Dash Application**

In [10]:
# Initialize the Dash app
app = dash.Dash(__name__)

**Step 3: Load the data**

In [11]:
sales_data = pd.read_csv('sales_data_500.csv')
print(sales_data.head())

         date product_id  sales_amount store_location
0  2023-01-01       P005           678          Miami
1  2023-01-01       P003           508    Los Angeles
2  2023-01-01       P007           320          Miami
3  2023-01-01       P010           985        Chicago
4  2023-01-01       P003           655        Houston


**Step 4: Create the Layout for the Dashboard**

In [13]:
# Create the layout for the Sales Dashboard
app.layout = html.Div([
    # Header
    html.H1("Sales Dashboard", style={'textAlign': 'center'}),

    # Description
    html.Div("Use the controls below to explore sales data interactively.", style={'textAlign': 'center'}),

    # Dropdown for selecting a store location
    html.Div([
        html.Label("Select Store Location:"),
        dcc.Dropdown(
            id='store-location-dropdown',
            options=[
                {'label': location, 'value': location} 
                for location in sales_data['store_location'].unique()
            ],
            value=sales_data['store_location'].unique()[0],  # Default value
        ),
    ], style={'width': '50%'}),

    # Slider for selecting a sales range
    html.Div([
        html.Label("Select Sales Range:"),
        dcc.RangeSlider(
            id='sales-slider',
            min=sales_data['sales_amount'].min(),
            max=sales_data['sales_amount'].max(),
            step=10,
            marks={int(sale): str(int(sale)) for sale in range(
                int(sales_data['sales_amount'].min()), 
                int(sales_data['sales_amount'].max()) + 100, 
                100
            )},
            value=[sales_data['sales_amount'].min(), sales_data['sales_amount'].max()]  # Default range
        )
    ], style={'width': '50%'}),

    # Graphs
    html.Div([
        dcc.Graph(id='sales-bar-chart'),  # Bar chart placeholder
        dcc.Graph(id='sales-line-chart')  # Line chart placeholder
    ])
])


**Step 5: Create the Callback Functions for Interactivity**

In [14]:
@app.callback(
    [Output('sales-bar-chart', 'figure'),
     Output('sales-line-chart', 'figure')],
    [Input('store-location-dropdown', 'value'),
     Input('sales-slider', 'value')]
)
def update_graphs(selected_location, sales_range):
    # Filter the dataset based on selected store location and sales range
    filtered_df = sales_data[
        (sales_data['store_location'] == selected_location) & 
        (sales_data['sales_amount'] >= sales_range[0]) & 
        (sales_data['sales_amount'] <= sales_range[1])
    ]

    # Handle empty dataset
    if filtered_df.empty:
        no_data_figure = px.bar(
            title="No Data Available",
            labels={'sales_amount': 'Sales Amount', 'store_location': 'Store Location'}
        )
        return no_data_figure, no_data_figure

    # --- Bar Chart: Total Sales by Product ---
    sales_by_product = (
        filtered_df.groupby('product_id')['sales_amount']
        .sum()
        .reset_index()
    )
    bar_chart = px.bar(
        sales_by_product,
        x='product_id',
        y='sales_amount',
        title=f"Total Sales by Product in {selected_location}",
        labels={'product_id': 'Product ID', 'sales_amount': 'Sales Amount'}
    )

    # --- Line Chart: Sales Trends Over Time ---
    sales_trends = (
        filtered_df.groupby('date')['sales_amount']
        .sum()
        .reset_index()
    )
    line_chart = px.line(
        sales_trends,
        x='date',
        y='sales_amount',
        title=f"Sales Trends Over Time in {selected_location}",
        labels={'date': 'Date', 'sales_amount': 'Sales Amount'}
    )

    return bar_chart, line_chart


**Step 7: Run the App**

In [16]:
if __name__ == '__main__':
    app.run_server(debug=True, port=8070)
